In [11]:
%reset -f

In [5]:
import shutil
import pandas as pd
import os

bigparFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\CLIGEN\2015parfiles\2015parfiles\2015parfiles'
parFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_parfiles'
outFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_tseries'
coordFILE = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\station_info.csv'

print(coordFILE)
df = pd.read_csv(coordFILE)
print(df)
parstation_list = df['stationID_left'].values

for stationID in parstation_list:
  shutil.copyfile(os.path.join(bigparFOLDER, stationID + '.par'), os.path.join(parFOLDER, stationID + '.par'))
    


C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\station_info.csv
    index stationID_left  long_left  ...      dist        dist_m  complete_per
0       1       nv264436    -115.17  ...  0.010512   1082.606551    100.000000
1       2       tx412121    -101.22  ...  0.058800   5493.501337     99.863107
2       3       or354811    -122.68  ...  0.008713    705.957843     99.733060
3       4       wi470516     -89.73  ...  0.012104   1323.876901     99.671458
4       5       il110510     -89.85  ...  0.010530   1146.757869     99.370294
5       6       id104670    -114.53  ...  0.016519   1646.391077     99.212868
6       7       ar032443     -94.17  ...  0.010439   1153.110087     99.185489
7       8       oh336405     -83.95  ...  0.035765   3885.095840     98.945927
8       9       az028619    -110.07  ...  0.008220    907.892738     98.501027
9      10       ut425733    -109.55  ...  0.007000    721.376590     98.069815
10     11       ms221707     -90.57  ...  0.018676   

In [ ]:
#==============================================================================
# Runs CLIGEN
#==============================================================================

from shutil import copyfile
import subprocess as sub
import os

# =============================================================================
# Manually enter directory paths below
# =============================================================================
cligenFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\cligenv53'
parFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_parfiles'
outFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_tseries'
consFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_cons'

REC_LEN = 40

parFiles = os.listdir( parFOLDER )


def run_cligen( parFile, cligenDir, parDir, outputDir, consoleDir ):

    error_file = []

    # =========================================================================
    # copy parfile to cligen directory
    # =========================================================================
    copyfile( os.path.join( parDir, parFile ), os.path.join( cligenDir, parFile ) )

    args = 'cligen53 -b1 -y{} -t5 -i{} -o{}'.format( REC_LEN, parFile, parFile[:-4] + '.txt' )
    with open(os.path.join( consoleDir, parFile[:-4] + '.txt' ), 'w') as f_out:
    
        try:
            result = sub.run( args=args, check=True, capture_output=True, text=True, shell=True, cwd=cligenDir )
            # =========================================================================
            # remove parfile from cligen directory after executing
            # =========================================================================
            os.remove( os.path.join( cligenDir, parFile ) )
        
            # =========================================================================
            # copy output to output folder and remove output in cligen directory
            # =========================================================================
            copyfile( os.path.join( cligenDir, parFile[:-4] + '.txt' ), 
                      os.path.join( outputDir, parFile[:-4] + '.txt' ) )
        
            os.remove( os.path.join( cligenDir, parFile[:-4] + '.txt' ) )
        
            f_out.write( result.stdout )
            f_out.write( result.stderr )
        
        except sub.CalledProcessError:
            # =========================================================================
            # remove parfile from cligen directory after executing
            # =========================================================================
            os.remove( os.path.join( cligenDir, parFile ) )
                    
            error_file.append( parFile )

    return error_file


run_ct = 0
error_files = []
for file in parFiles:
    output = run_cligen( file, cligenFOLDER, parFOLDER, outFOLDER, consFOLDER )
    run_ct += 1
    print(str(run_ct) + ' / ' + str(len(parFiles)))
    error_files.extend(output)

print('error files:\n', error_files)

In [20]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import os

cligenFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\cligenv53'
parFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_parfiles'
tseriesFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_tseries'
grndFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_GHCN_records'
prnFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_prn_records'
keyFILE = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\station_info.csv'

key_df = pd.read_csv(keyFILE)
key_df = key_df.loc[(key_df['complete_per'] > 95.0),:]
print(key_df)

dates = []
start_date = dt(1974, 1, 1, 0, 0)
end_date = dt(2013, 12, 31, 0, 0)
date = start_date
while date <= end_date:
  dates.append(date)
  date = date + timedelta(days=1)

for fclg in key_df['stationID_left'].values:
  prnFILE = os.path.join(prnFOLDER, fclg + '.prn')
  cligenFILE = os.path.join(tseriesFOLDER, fclg + '.txt')
  grndID = key_df.loc[(key_df['stationID_left'] == fclg), 'stationID_right'].values[0]
  grndFILE = os.path.join(grndFOLDER, grndID + '.csv')
  grnd_df = pd.read_csv(grndFILE)
  precip_list = grnd_df[grndID]

  with open(cligenFILE) as f:
    lines = f.readlines()

  tmax_tseries = []
  tmin_tseries = []
  for line in lines[15:-1]:
    tmax = float(line[35:41])
    tmin = float(line[41:47])
    tmax = str(int(round((tmax * (9./5.)) + 32, 0)))
    tmin = str(int(round((tmin * (9./5.)) + 32, 0)))
    tmax_tseries.append(tmax)
    tmin_tseries.append(tmin)

  with open(prnFILE, 'w') as fo:
    for i, date in enumerate(dates):
      day = str(date.day)
      month = str(date.month)
      year = str(date.year)
      prcp = str(int(round(precip_list[i]/.254, 0)))
      s = month.rjust(5) + day.rjust(5) + year.rjust(5) + prcp.rjust(5) + tmax_tseries[i].rjust(5) + tmin_tseries[i].rjust(5) + '\n'
      fo.write(s)




    index stationID_left  long_left  ...      dist       dist_m  complete_per
0       1       nv264436    -115.17  ...  0.010512  1082.606551    100.000000
1       2       tx412121    -101.22  ...  0.058800  5493.501337     99.863107
2       3       or354811    -122.68  ...  0.008713   705.957843     99.733060
3       4       wi470516     -89.73  ...  0.012104  1323.876901     99.671458
4       5       il110510     -89.85  ...  0.010530  1146.757869     99.370294
5       6       id104670    -114.53  ...  0.016519  1646.391077     99.212868
6       7       ar032443     -94.17  ...  0.010439  1153.110087     99.185489
7       8       oh336405     -83.95  ...  0.035765  3885.095840     98.945927
8       9       az028619    -110.07  ...  0.008220   907.892738     98.501027
9      10       ut425733    -109.55  ...  0.007000   721.376590     98.069815
10     11       ms221707     -90.57  ...  0.018676  1915.169804     96.906229
11     12       ia133438     -94.47  ...  0.035511  3519.294914 

In [23]:

#==============================================================================
# Runs CLIGEN
#==============================================================================

from shutil import copyfile
import subprocess as sub
import os

# =============================================================================
# Manually enter directory paths below
# =============================================================================
cligenFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\cligenv53'
parFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_parfiles'
tseriesFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_prn_records'
outFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_Semisto_tseries'
consFOLDER = r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_cons'
prnFOLDER =  r'C:\Users\afullhart\Google Drive\Colab Notebooks\TeamProjects\15_USNET_prn'

REC_LEN = 40

parFiles = os.listdir( parFOLDER )

def run_cligen( parFile, cligenDir, parDir, tseriesDir, outputDir, consoleDir ):
    print(parFile)
    error_file = []

    # =========================================================================
    # copy parfile to cligen directory
    # =========================================================================
    copyfile( os.path.join( parDir, parFile ), os.path.join( cligenDir, parFile ) )
    copyfile( os.path.join( tseriesDir, parFile[:-4] + '.prn' ), os.path.join( cligenDir, parFile[:-4] + '.prn' ) )

    args = 'cligen53 -b1974 -y{} -t6 -i{} -o{} -O{}'.format( REC_LEN, parFile, parFile[:-4] + '_semi.txt', parFile[:-4] + '.prn' )
    with open(os.path.join( consoleDir, parFile[:-4] + '.txt' ), 'w') as f_out:
    
        try:
            result = sub.run( args=args, check=True, capture_output=True, text=True, shell=True, cwd=cligenDir )
            # =========================================================================
            # remove parfile from cligen directory after executing
            # =========================================================================
            os.remove( os.path.join( cligenDir, parFile ) )
            os.remove( os.path.join( cligenDir, parFile[:-4] + '.prn' ) )       
            # =========================================================================
            # copy output to output folder and remove output in cligen directory
            # =========================================================================
            copyfile( os.path.join( cligenDir, parFile[:-4] + '_semi.txt' ), 
                      os.path.join( outputDir, parFile[:-4] + '_semi.txt' ) )
        
            os.remove( os.path.join( cligenDir, parFile[:-4] + '_semi.txt' ) )
        
            f_out.write( result.stdout )
            f_out.write( result.stderr )
        
        except sub.CalledProcessError:
            # =========================================================================
            # remove parfile from cligen directory after executing
            # =========================================================================
            os.remove( os.path.join( cligenDir, parFile ) )
            os.remove( os.path.join( cligenDir, parFile[:-4] + '.prn' ) )
            os.remove( os.path.join( cligenDir, parFile[:-4] + '_semi.txt' ) )
            error_file.append( parFile )
            
    return error_file



run_ct = 0
error_files = []
for file in parFiles:
    print(file)
    output = run_cligen( file, cligenFOLDER, parFOLDER, tseriesFOLDER, outFOLDER, consFOLDER )
    run_ct += 1
    print(str(run_ct) + ' / ' + str(len(parFiles)))
    error_files.extend(output)

print('error files:\n', error_files)




ar032443.par
ar032443.par
1 / 15
az028619.par
az028619.par
2 / 15
ca040449.par
ca040449.par
3 / 15
fl081544.par
fl081544.par
4 / 15
ia133438.par
ia133438.par
5 / 15
id104670.par
id104670.par
6 / 15
il110510.par
il110510.par
7 / 15
ms221707.par
ms221707.par
8 / 15
nv264436.par
nv264436.par
9 / 15
oh336405.par
oh336405.par
10 / 15
or354811.par
or354811.par
11 / 15
tx412121.par
tx412121.par
12 / 15
ut425733.par
ut425733.par
13 / 15
wa454679.par
wa454679.par
14 / 15
wi470516.par
wi470516.par
15 / 15
error files:
 []
